[<a id="top"></a>

### Part 2:  Movie Ratings Data using NMF 

Complete Part 2. 

Limitation(s) of sklearn’s non-negative matrix factorization library. [20 pts]

1. Load the movie ratings data (as in the HW3-recommender-system) and use matrix factorization technique(s) and predict the missing ratings from the test data. Measure the RMSE. You should use sklearn library. [10 pts]


2. Discuss the results and why sklearn's non-negative matrix facorization library did not work well compared to simple baseline or similarity-based methods we’ve done in Module 3. Can you suggest a way(s) to fix it? [10 pts]


### >>> [Summary of results here](#summary)

#### 1) Load movie ratings data from HW3

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error
from math import sqrt

import os
from collections import namedtuple

In [2]:
path = '/Users/ayeshamendoza/UCB/5510 - Unsupervised Learning'
MV_users = pd.read_csv(os.path.join(path, 'users.csv'))
MV_movies = pd.read_csv(os.path.join(path, 'movies.csv'))
train = pd.read_csv(os.path.join(path, 'train.csv'))
test = pd.read_csv(os.path.join(path, 'test.csv'))

In [3]:
Data = namedtuple('Data', ['users','movies','train','test'])
data = Data(MV_users, MV_movies, train, test)

In [4]:
data.train.head()

,uID,mID,rating
0,744,1210,5
1,3040,1584,4
2,1451,1293,5
3,5455,3176,2
4,2507,3074,5


In [5]:
# Load data
# data = pd.read_csv('movie_ratings.csv')  # Replace with the path to your dataset
data_df = data.train.copy()

# Create a user-item matrix
user_item_matrix = data_df.pivot(index='uID', columns='mID', values='rating')

Implement NMF to predict movie ratings for users

In [6]:
data_df = data.train.copy()

# Create a user-item matrix
user_item_matrix = data_df.pivot(index='uID', columns='mID', values='rating')

# Impute missing values with 0 for training purposes
user_item_matrix_filled = user_item_matrix.fillna(0)

# Split data into train and test sets
train_data, test_data = train_test_split(data_df, test_size=0.2, random_state=42)

# Create training matrix with only training data
train_matrix = user_item_matrix.copy()
for row in test_data.itertuples():
    train_matrix.at[row.uID, row.mID] = np.nan

train_matrix = train_matrix.fillna(0)

# Matrix Factorization using NMF
nmf = NMF(n_components=50, init='random', random_state=42)
user_factors = nmf.fit_transform(train_matrix)
item_factors = nmf.components_

# Predict missing ratings
predicted_matrix = np.dot(user_factors, item_factors)

# Re-create the full matrix with predicted values
predicted_ratings = pd.DataFrame(predicted_matrix, index=user_item_matrix.index, columns=user_item_matrix.columns)

# Extract the predicted ratings for the test set
test_predictions = []
test_actuals = []
for row in test_data.itertuples():
    predicted_rating = predicted_ratings.at[row.uID, row.mID]
    test_predictions.append(predicted_rating)
    test_actuals.append(row.rating)

# Calculate RMSE
rmse = sqrt(mean_squared_error(test_actuals, test_predictions))
print(f'RMSE: {rmse}')

/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1076: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


RMSE: 3.1004066862406874


[<a id="summary"></a>
### 2)  Results Summary:

It appears that NMF does not perform as well as the content-based and collaborative filtering techniques used in Week 3 for the movie ratings data possibly due to the data sparsity issue.  As we have noticed the movie ratings data is very sparse, and NMF requires a dense matrix in order to effectively decompose a matrix to lower dimension matrices.

Techniques such as Collaborative filtering are more straightforward where we directly compare user-item interactions, which could work better and be interpreted easier than the latent features/factors created from NMF.

Some ways that we can fix or improve this is would be:
- We can impute values for missing ratings this can help create a more dense matrix and address sparsity issue.
- More extensive hyperparameter tuning can be done to help improve the performance of NMF
- I have read some hybrid approaches can be done, where features generated from NMF can be used for collaborative filtering.

[Back to Top](#top)